# Получение текстов для Sentiment Analysis

## Задача

Получение размеченных текстов из открытого источника

## Данные

В качестве таковых текстов взяты отзывы на фильмы и сериалы с сайта [Кинопоиск](https://www.kinopoisk.ru/lists/movies/) с 2020 по 2023 год включительно

## Расчёты

In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm.notebook import tqdm
import re
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# Каждый фильм/сериал на сайте имеет уникальный номер
# Получим эти самые номера фильмов
film_number = []

browser = Chrome()

start_year = 2020
end_year = 2023

for year in tqdm(range(start_year, end_year + 1)):

    # Отдельно рассматривается каждый год и список фильмов сортируется по количеству оценок пользователей
    url = f"https://www.kinopoisk.ru/lists/movies/year--{year}/?sort=votes"

    browser.get(url)


    while True:

        soup = BeautifulSoup(browser.page_source, 'lxml')

        # На одной странице находится 50 фильмов/сериалов
        for number in soup.find_all('div', class_='styles_main__Y8zDm'):
            # Добавление всех номеров фильмов/сериалов на странице
            film_number.append(re.search(r'/\d+/', number.a['href'])[0])

        # Нажатие на кнопу "Вперёд" для переключения на следующую страницу
        next_button = browser.find_elements(by=By.CLASS_NAME, value="styles_end__aEsmB")

        # Если у всех фильмов на странице отсутствуют оценки, то дальнейший просмотри страниц данного года прекращается
        # Предположительно, если отсутствутствуют оценки, то отзывы и подавно
        if next_button and ([i.text for i in soup.find_all('div', class_='styles_rating__LU3_x')] != ['—'] * 50):
            next_button[0].click()
            WebDriverWait(browser, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "styles_main__Y8zDm")))
        else:
            break

  0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
# Количество найденных фильмов/сериалов
len(film_number)

17600

In [4]:
# Удаление повторяющихся названий в списке
# Так как сериалы, из-за своей продолжительности, могут попасть в список за разные годы
film_number = list(set(film_number))
len(film_number)

13285

In [5]:
# Получение отзывов на фильмы/сериалы по полученным номерам
# Запись отзывов в файл data/reviews_2020_2023.csv
reviews = pd.DataFrame(columns=["type", "review"])
reviews.to_csv(f'data/reviews_{start_year}_{end_year}.csv', sep='|', mode='w', header=True, index=False, encoding='utf-8')

number_of_reviews = 0

for number in tqdm(film_number):

    browser.get(f'https://www.kinopoisk.ru/film{number}reviews/ord/date/status/all/perpage/200/')

    while True:

        soup = BeautifulSoup(browser.page_source, 'lxml')

        for review in soup.find_all('div', class_='reviewItem userReview'):
            
            review = review.find('div', class_='response')
        
            reviews.loc[len(reviews.index)] = [
                review["class"][1],
                re.sub('[\n| ]+', ' ', review.table.tbody.tr.text).strip()
            ]

            number_of_reviews += 1

        reviews.to_csv(f'data/reviews_{start_year}_{end_year}.csv', sep='|', mode='a', header=False, index=False, encoding='utf-8')
        reviews.drop(labels=reviews.index, axis=0, inplace=True)

        next_button = browser.find_elements(by=By.LINK_TEXT, value="»")
        if next_button:
            next_button[0].click()
            WebDriverWait(browser, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "response")))
        else:
            break

print('Общее число отзывов:', number_of_reviews)

  0%|          | 0/13285 [00:00<?, ?it/s]

Общее число отзывов: 48790


## Результаты

Было получено больше 48 тысяч отзывов с пометками (негативный, нейтральный, положительный) на фильмы или сериалы за почти 4 года (с 2020 по 2023 год).